In [2]:
# GET LIBRARIES
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import csv


In [5]:
# DEFINE FUNCTIONS

def load_suite2p_outputs(directory_path): # this will load all Suite2p output files from the specified directory
    file_dict = {
        'roi_fluorescence': '*F.npy',
        'neuropil_fluorescence': '*Fneu.npy',
        'cell_identifier' : '*iscell.npy',
        'intermediate_outputs' : '*ops.npy',
        'roi_traces' : '*spks.npy',
        'roi_statistics' : '*stat.npy'}
    loaded_data_files = {}
    pathlist = Path(directory_path)
    for key, value in file_dict.items():
        suite2p_files = list(pathlist.glob(value))
        for file in suite2p_files:
            loaded_data_files[key] = np.load(file, allow_pickle = True)
    return loaded_data_files

def calculate_dff(raw, baseline): # this will calculate the dff values for each roi
    dff_list = []
    for row in range(len(raw)):
        dff = ((raw[row] - baseline[row])/(baseline[row]))
        dff_list.append(dff)
    return dff_list

def calculate_baseline(raw_fluorescence, percentile): # this will calculate the specified percentile along each roi's raw fluorescence
    percentile_list = []
    for row in raw_fluorescence:
        find_percentile = np.percentile(row, percentile, keepdims = True)
        percentile_list.append(find_percentile)
    return percentile_list

def export_dff_to_csv(dff_data, output_path):
    # Convert the list of ΔF/F arrays into a DataFrame
    dff_df = pd.DataFrame(dff_data)
    
    # Export to CSV
    dff_df.to_csv(output_path, index=False)
    print(f"ΔF/F data successfully exported to {output_path}")

In [ ]:
# LOAD DATA
suite2p_data_output = load_suite2p_outputs(r'C:\dev\2p-analysis\func\SB05\tiff\sb05_high\suite2p\plane0')

print(suite2p_data_output.keys())

In [ ]:
# FILTER ROIs (CELLS ONLY)

# assign bool T/F based on confidence index
true_cells_only = suite2p_data_output['cell_identifier'][:,0].astype(bool)
print(true_cells_only)

# filter ROIs and neuropil based on bool value
filtered_roi = np.array(suite2p_data_output['roi_fluorescence'][true_cells_only])
filtered_neuropil = np.array(suite2p_data_output['neuropil_fluorescence'][true_cells_only])

In [12]:
# NEUROPIL SUBTRACTION

neuropil_subtracted_roi = (filtered_roi - (0.7*filtered_neuropil)) #TODO: necessary?

In [ ]:
# PLOT ROIS
plt.plot(neuropil_subtracted_roi[5])
plt.title('Neuropil Subtracted Fluorescence of ROI')
plt.xlabel('frames')
plt.ylabel('raw fluorescence')
plt.show()

In [ ]:
# CALCULATE BASELINE FLUORESCENCE
baseline_fluorescence = np.array(calculate_baseline(filtered_roi, percentile = 10))
print(baseline_fluorescence)

In [ ]:
# CALCULATE DF/F
roi_dff = calculate_dff(filtered_roi, baseline_fluorescence)
print(roi_dff)

In [44]:
# SET SESSION VARIABLES (NUM ROIS, FRAMES, SESS DURATION, FRAME RATE)

number_rois = len(filtered_roi)
number_frames = len((filtered_roi[:,0:(6000-81)])[0]) #len[0] yields the second dimension of array; 91 is the offset amount of frames
neuro_frame_rate = 9.865
session_dur = number_frames/neuro_frame_rate

print(number_rois)
print(number_frames)
print(session_dur)

60
5919
600.0


In [48]:
# IMPORT PSYCHOPY TIMESTAMPS
# Define the path to the CSV file
path = r"C:\dev\sipefield-gratings\PsychoPy\data\high\sub-SB03\ses-01\beh\sub-SB03_ses-01_20240808_152511.csv"

# Load the CSV file into a pandas DataFrame
psychopy_trial_timestamps = pd.read_csv(path)

# Display the first few rows of the DataFrame
#print(psychopy_trial_timestamps.head())

# List the columns from the DataFrame
print(psychopy_trial_timestamps.columns)



Index(['trials.thisRepN', 'trials.thisTrialN', 'trials.thisN',
       'trials.thisIndex', 'thisRow.t', 'notes', 'nidaqTrigger.started',
       'text_waiting_message.started', 'key_resp.started',
       'nidaqTrigger.stopped', 'key_resp.keys', 'key_resp.rt',
       'key_resp.duration', 'display_gratings.started',
       'stim_grayScreen.started', 'stim_grating.started',
       'stim_grayScreen.stopped', 'stim_grating.stopped',
       'display_gratings.stopped', 'Protocol ID', 'Subject ID', 'Session ID',
       'Wait for Trigger', 'date', 'expName', 'psychopyVersion', 'frameRate',
       'expStart', 'Unnamed: 28'],
      dtype='object')


In [52]:
# EXTRACT TIMESTAMPS FROM PSYCHOPY OUTPUT
trial_start = psychopy_trial_timestamps['display_gratings.started']
grating_on = psychopy_trial_timestamps['stim_grating.started']
trial_end = psychopy_trial_timestamps['display_gratings.stopped']

trial = enumerate(zip(trial_start, grating_on, trial_end))
print(trial)


TypeError: object of type 'enumerate' has no len()